# AWS Lambda FaaS Deployment with Hydraa

This notebook demonstrates how to deploy and manage serverless functions on AWS Lambda using FaaS Manager.

## Setup and Initialization

In [ ]:
import os
import json
import time
import traceback
from hydraa import proxy, AWS, Task
from hydraa.services import ServiceManager
from hydraa_faas.faas_manager.manager import FaasManager

In [ ]:
# set aws credentials (replace with your actual credentials)
# in production you would use iam roles or environment variables
os.environ["ACCESS_KEY_ID"] = "YOUR_ACCESS_KEY_ID"
os.environ["ACCESS_KEY_SECRET"] = "YOUR_SECRET_ACCESS_KEY"
os.environ["AWS_REGION"] = "us-east-1"

# resource configuration options
resource_config = {
    'aws': {
        'auto_setup_resources': True,  # automatically create iam role
        'enable_container_support': False  # set to true if deploying container images
    }
}

# alternative: use existing resources
# resource_config = {
#     'aws': {
#         'use_existing': {
#             'iam_role_arn': 'arn:aws:iam::123456789:role/existing-lambda-role',
#             'ecr_repository_uri': '123456789.dkr.ecr.us-east-1.amazonaws.com/my-repo'
#         },
#         'enable_container_support': True
#     }
# }

# initialize proxy and faas manager
provider_mgr = proxy([AWS])

try:
    faas_mgr = FaasManager(
        proxy_mgr=provider_mgr,
        asynchronous=True,
        auto_terminate=True,  # clean up functions on shutdown
        resource_config=resource_config
    )

    # start services
    service_mgr = ServiceManager([faas_mgr])
    service_mgr.start_services()
    
    # check status
    status = faas_mgr.get_status()
    print("FaaS Manager Status:")
    print(json.dumps(status, indent=2))
    
except Exception as e:
    print(f"Failed to initialize FaaS Manager: {e}")
    traceback.print_exc()

## 3. Container Image Deployment (requires container support)

In [ ]:
# NOTE: this requires enable_container_support=True in resource_config
# and either an existing ecr repository or auto_setup_resources=True

# build and deploy from source with container
container_build_task = Task()
container_build_task.provider = 'lambda'
container_build_task.memory = 2048
container_build_task.vcpus = 2
container_build_task.timeout = 600  # 10 minutes
container_build_task.source_path = './lambda_functions/ml_inference'
container_build_task.build_image = True  # Build and push to ECR

# uncomment to deploy (requires container support enabled)
# try:
#     print("Building and deploying container function...")
#     faas_mgr.submit(container_build_task)
#     result = container_build_task.result()
#     print(f"Container function deployed: {result}")
# except Exception as e:
#     print(f"Container deployment failed: {e}")
#     print("Enable container support in resource_config to use this feature")

In [ ]:
# deploy from existing container image
# container_task = Task()
# container_task.provider = 'lambda'
# container_task.memory = 2048
# container_task.vcpus = 2
# container_task.timeout = 600
# container_task.image = '123456789.dkr.ecr.us-east-1.amazonaws.com/ml-model:latest'
# 
# # uncomment to deploy (requires valid ECR image URI)
# try:
#     faas_mgr.submit(container_task)
#     result = container_task.result()
#     print(f"Container function deployed: {result}")
# except Exception as e:
#     print(f"Container deployment failed: {e}")

## Advanced Features

In [ ]:
# event driven function with s3 trigger configuration
s3_task = Task()
s3_task.provider = 'lambda'
s3_task.memory = 512
s3_task.vcpus = 1
s3_task.timeout = 60
s3_task.runtime = 'python3.9'
s3_task.handler = 's3_handler.lambda_handler'
s3_task.handler_code = '''
import json
import boto3
import urllib.parse

s3 = boto3.client('s3')

def lambda_handler(event, context):
    # Process S3 event
    for record in event.get('Records', []):
        # Get the object from the event
        bucket = record['s3']['bucket']['name']
        key = urllib.parse.unquote_plus(record['s3']['object']['key'], encoding='utf-8')
        
        try:
            # Get object metadata
            response = s3.head_object(Bucket=bucket, Key=key)
            
            print(f"Processing file: {key} from bucket: {bucket}")
            print(f"File size: {response['ContentLength']} bytes")
            
            # Add processing logic here
            
            return {
                'statusCode': 200,
                'body': json.dumps({
                    'message': f'Successfully processed {key}',
                    'bucket': bucket,
                    'size': response['ContentLength']
                })
            }
            
        except Exception as e:
            print(f"Error processing object {key}: {str(e)}")
            raise e
'''

try:
    # deploy s3 processing function
    print("Deploying S3 processing function...")
    faas_mgr.submit(s3_task)
    result = s3_task.result()
    print(f"S3 function deployed: {result}")
    print("Note: You'll need to manually configure S3 trigger in AWS Console")
except Exception as e:
    print(f"S3 function deployment failed: {e}")
    traceback.print_exc()

In [ ]:
# batch deployment of related functions
batch_tasks = []
function_types = ['validator', 'processor', 'notifier']

for func_type in function_types:
    task = Task()
    task.provider = 'lambda'
    task.memory = 256
    task.vcpus = 1
    task.timeout = 30
    task.runtime = 'python3.9'
    task.handler = f'{func_type}.handler'
    task.handler_code = f'''
import json
import time

def handler(event, context):
    function_type = '{func_type}'
    
    # Simulate processing
    processing_time = 0.5
    time.sleep(processing_time)
    
    return {{
        'statusCode': 200,
        'body': json.dumps({{
            'function': function_type,
            'processed': True,
            'timestamp': int(time.time()),
            'request_id': context.request_id
        }})
    }}
'''
    task.env_vars = {
        'FUNCTION_TYPE': func_type,
        'STAGE': 'production'
    }
    batch_tasks.append(task)

try:
    # deploy batch
    print("Deploying batch of functions...")
    faas_mgr.submit(batch_tasks)
    
    for i, task in enumerate(batch_tasks):
        result = task.result()
        print(f"{function_types[i]} deployed: {task.name}")
    
    # test orchestration by invoke functions in sequence
    print("\nTesting function orchestration...")
    data = {'input': 'test_data'}
    
    for task, func_type in zip(batch_tasks, function_types):
        response = faas_mgr.invoke(task.name, data, provider='lambda')
        print(f"{func_type} response: {response.get('Payload', {})[:100]}...")
        # pass output to next function
        if response.get('StatusCode') == 200:
            data = json.loads(response.get('Payload', '{}'))
            
except Exception as e:
    print(f"Batch deployment failed: {e}")
    traceback.print_exc()

In [ ]:
# vpc enabled function for private resources
vpc_task = Task()
vpc_task.provider = 'lambda'
vpc_task.memory = 1024
vpc_task.vcpus = 1
vpc_task.timeout = 60
vpc_task.runtime = 'python3.9'
vpc_task.handler = 'vpc_handler.handler'
vpc_task.handler_code = '''
import json
import boto3
import os

def handler(event, context):
    # This function can access private VPC resources
    # like RDS databases, ElastiCache, private EC2 instances
    
    vpc_config = {
        'in_vpc': True,
        'security_groups': os.environ.get('AWS_LAMBDA_SECURITY_GROUP_IDS', '').split(','),
        'subnets': os.environ.get('AWS_LAMBDA_SUBNET_IDS', '').split(',')
    }
    
    # Example: Connect to private RDS
    # db_host = os.environ.get('DB_HOST')
    # db_name = os.environ.get('DB_NAME')
    
    return {
        'statusCode': 200,
        'body': json.dumps({
            'message': 'VPC-enabled function executed',
            'vpc_config': vpc_config
        })
    }
'''

# configure vpc (replace with your actual vpc configuration)
# vpc_task.vpc_config = {
#     'SubnetIds': ['subnet-12345', 'subnet-67890'],
#     'SecurityGroupIds': ['sg-12345']
# }

# uncomment to deploy (requires valid VPC configuration)
# try:
#     print("Deploying VPC-enabled function...")
#     faas_mgr.submit(vpc_task)
#     result = vpc_task.result()
#     print(f"VPC function deployed: {result}")
# except Exception as e:
#     print(f"VPC deployment failed: {e}")
#     traceback.print_exc()

In [ ]:
# list and manage functions
try:
    functions = faas_mgr.list_functions(provider='lambda')
    print("Deployed Lambda functions:")
    for provider, funcs in functions.items():
        print(f"\nProvider: {provider}")
        for func in funcs:
            print(f"  - {func['name']}")
            print(f"    Runtime: {func.get('runtime', 'N/A')}")
            print(f"    Memory: {func.get('memory', 0)} MB")
            print(f"    Timeout: {func.get('timeout', 0)} seconds")
            print(f"    State: {func.get('state', 'unknown')}")
            print(f"    Package Type: {func.get('package_type', 'Zip')}")
            print(f"    ARN: {func.get('arn', 'N/A')}")
            print()

    # get manager status
    status = faas_mgr.get_status()
    print(f"\nManager status: {json.dumps(status, indent=2)}")
except Exception as e:
    print(f"Failed to list functions: {e}")
    traceback.print_exc()

## Performance Testing

In [ ]:
# test function performance and cold starts
perf_task = Task()
perf_task.provider = 'lambda'
perf_task.memory = 512
perf_task.vcpus = 1
perf_task.timeout = 10
perf_task.runtime = 'python3.9'
perf_task.handler = 'perf.handler'
perf_task.handler_code = '''
import json
import time
import os

# Global variable to detect cold starts
cold_start = True
init_time = time.time()

def handler(event, context):
    global cold_start
    
    start_time = time.time()
    is_cold_start = cold_start
    cold_start = False
    
    # Simulate some work
    work_duration = event.get('work_duration', 0.1)
    time.sleep(work_duration)
    
    execution_time = (time.time() - start_time) * 1000  # ms
    
    return {
        'statusCode': 200,
        'body': json.dumps({
            'cold_start': is_cold_start,
            'execution_time_ms': round(execution_time, 2),
            'memory_size': context.memory_limit_in_mb,
            'remaining_time_ms': context.get_remaining_time_in_millis(),
            'request_id': context.request_id
        })
    }
'''

try:
    print("Deploying performance test function...")
    faas_mgr.submit(perf_task)
    result = perf_task.result()
    print(f"Performance function deployed: {result}")
    
    time.sleep(5)
    
    # run performance tests
    print("\nRunning performance tests...")
    for i in range(5):
        response = faas_mgr.invoke(perf_task.name, {'work_duration': 0.1}, provider='lambda')
        if response.get('StatusCode') == 200:
            payload = json.loads(response.get('Payload', '{}'))
            body = json.loads(payload.get('body', '{}'))
            print(f"Test {i+1}: Cold start: {body.get('cold_start')}, "
                  f"Execution time: {body.get('execution_time_ms')} ms")
        time.sleep(1)  # small delay between invocations
        
except Exception as e:
    print(f"Performance test failed: {e}")
    traceback.print_exc()

## Cleanup

In [ ]:
# delete specific function
# try:
#     if 'inline_task' in locals() and hasattr(inline_task, 'name'):
#         faas_mgr.delete_function(inline_task.name, provider='lambda')
#         print(f"Deleted function: {inline_task.name}")
# except Exception as e:
#     print(f"Failed to delete function: {e}")

# shutdown manager (auto_terminate=True will cleanup all functions)
# resources created by this instance will also be cleaned up
# try:
#     print("Shutting down FaaS manager...")
#     service_mgr.shutdown_services()
#     print("Shutdown complete")
# except Exception as e:
#     print(f"Failed to shutdown: {e}")

## Advanced Resource Configuration Examples

In [ ]:
# example configurations for different scenarios

# example 1: full container support with ecr
container_config = {
    'aws': {
        'auto_setup_resources': True,
        'enable_container_support': True
    }
}

# example 2: bring your own everything
byo_config = {
    'aws': {
        'use_existing': {
            'iam_role_arn': 'arn:aws:iam::123456789:role/my-lambda-role',
            'ecr_repository_uri': '123456789.dkr.ecr.us-east-1.amazonaws.com/my-repo'
        },
        'auto_setup_resources': False,
        'enable_container_support': True
    }
}

# example 3: minimal setup (zip functions only)
minimal_config = {
    'aws': {
        'auto_setup_resources': True,
        'enable_container_support': False
    }
}

# example 4: production setup with custom configuration
production_config = {
    'aws': {
        'auto_setup_resources': True,
        'enable_container_support': True,
        'default_timeout': 60,
        'default_memory': 512,
        'tags': {
            'Environment': 'Production',
            'ManagedBy': 'Hydraa',
            'Team': 'DevOps'
        }
    }
}

print("Resource configuration examples ready for use")